In [5]:
def draw_graph(xmesh,ymesh,topo_data):
    import numpy as np
    import matplotlib.pyplot as plt
    import cartopy.crs as ccrs
    fig = plt.figure( figsize = (5,2.5), dpi = 300 )
    proj = ccrs.Mercator(central_longitude=140)
    proj_cart = ccrs.PlateCarree()
    ax = plt.axes((0.1,0.1,0.8,0.8), projection=proj)
    ax.set_extent([120,160,25,50],crs=proj_cart)
    ax.coastlines()
    
    C = ax.contourf(xmesh,ymesh,topo_data,cmap="RdYlBu",transform=proj_cart)
    fig.colorbar(C)
    fig.savefig("2020年")
    plt.show()
    return

In [7]:
def inquire_grib_data(path):
    import pygrib                  # gribファイルの中身を見たい場合はinquire_grib_data(path)を実行                
    grbs = pygrib.open(path)
    for grb in grbs:
        print(grb)
    return
def read_grib_data(path,name=None,level=None):
    import numpy as np
    import pygrib                   # pygribは!pip3 install pygrib --userでインストール
    grbs = pygrib.open(path)

    if name != None:                # anl_surf125に対しては変数名を与える
        alines = grbs.select(name=name)
    elif level != None:             # anl_p125に対しては気圧面を与えるとその水平面データ
        alines = grbs.select(level=level)
    else:                           #                  気圧面を与えないと全３次元データ
        alines = grbs.select()

    lat, lon = alines[0].latlons()  # lonは経度、latは緯度データ: (ny,nx)の２次元格子です
    ny, nx = lat.shape
    nline = len(alines)
    gdata = np.empty( (nline,ny,nx), dtype = "f4" )
    levels = np.empty( (nline), dtype = "f4" )
    for iline, aline in enumerate(alines):
        gdata[iline,:,:] = aline.values[::-1,:]
        levels[iline] = aline["level"]

    return lon, lat[::-1], level, gdata 

In [10]:
import datetime as dt
import calendar as cal
import numpy as np
#nmonth=cal.monthrange(2020,1)[1]
#dts=[dt.datetime(2020,1,1,0)+dt.timedelta(hours=6*i) for i in range(nmonth*4)]


dts=[]
for m in range(12):
    month=m+1
    nmonth=cal.monthrange(2020,month)[1]
    dts.extend([dt.datetime(2020,month,1,0)+dt.timedelta(hours=6*i) for i in range(nmonth*4)])  #一年分の平均

slpm=np.zeros((len(dts),145,288)) #データの置き場所をつくる

for i,dt in enumerate(dts):
    fdir = "/mnt/hwmet-hdd01/Database/JRA55/anl_p125/"
    fname = "/anl_surf125."
    path=dt.strftime(fdir+"%Y%m"+fname+"%Y%m%d%H")
    lon,lat,_,SLP0=read_grib_data(path,name="Mean sea level pressure")
    slpm[i,:,:]=SLP0
#print(slpm)
#Mean=np.mean(slpm,axis=0)
#print(Mean.shape)
#draw_graph(lon,lat,Mean)
draw_graph(lon,lat)

TypeError: draw_graph() missing 1 required positional argument: 'topo_data'